# Caso práctico de Analítica Escalable (Ejercicios) #

En este notebook, se van a realizar los ejercicios del módulo. En lugar de tener contenido teórico y descripciones, se dejarán únicamente las celdas de código necesarias para su ejecución.

Para completar los ejercicios, hay que codificar y ejecutar la solución en las celdas que se encuentran justo debajo de los enunciados de los ejercicios.

Una vez se haya terminado, en el menú de la izquiera, a la hora de seleccionar el notebook, si se le hace click a la flecha que se encuentra en la derecha, se puede exportar al notebook. Hay que exportarlo en formato DBC (Databricks Notebook) como en HTML.

In [3]:
print(sc.version)

2.4.5

Los ejercicios consistirán en añadir nuevas funcionalidades, o ejecutar nuevo código, sobre el Notebook que contiene toda la teoría vista en el módulo. Por ello, gran parte del código que se encuentra dentro del notebook de contenido teórico se encontrará aquí de nuevo, pero se pedirá nuevo código.

## Importando los datos ##

In [6]:
dbutils.fs.cp("/FileStore/tables/Hotel_Reviews.csv", "file:///databricks/driver/Hotel_Reviews.csv")

Out[4]: True

In [7]:
def score_to_string(score):
  if score < 5:
    return "Bad"
  elif score < 7:
    return "Normal"
  elif score < 9:
    return "Good"
  elif score < 10: 
    return "Excellent"
  else:
    return "Perfect"
  
def score_to_evaluation(score_string):
  score_dict = {
    "Bad": 0,
    "Normal": 1,
    "Good": 2,
    "Excellent": 3,
    "Perfect": 4
  }
  return score_dict.get(score_string, None)

## DataFrames en Spark: SparkSQL. ##

In [9]:
df_spark_sql = spark.read.format("csv")\
         .option("header", "true")\
         .option("inferSchema", "true")\
         .load("/FileStore/tables/Hotel_Reviews.csv")

In [10]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, IntegerType

score_string_udf = udf(score_to_string, StringType())
score_evaluation_udf = udf(score_to_evaluation, IntegerType())

In [11]:
df_spark_sql = df_spark_sql.withColumn('score_string',score_string_udf(df_spark_sql["Average_Score"]))
df_spark_sql = df_spark_sql.withColumn('score_evaluation',score_evaluation_udf(df_spark_sql["score_string"]))

In [12]:
def day_to_int(day):
  return int(day.replace(" days", "").replace(" day", ""))
day_to_int_udf = udf(day_to_int, IntegerType())
df_spark_sql = df_spark_sql.withColumn("days_since_review", day_to_int_udf(df_spark_sql["days_since_review"]))

### Ejercicio 1: Crear un bucle que muestre todas las columnas del DataFrame, junto con sus tipos. También puedes pintar el esquema del Dataframe. ###

Voy a empezar creando un bucle con for "i" in df_spark_sql.dtypes e imprimiendo "i" en cada iteración. De esta forma nos imprime la información que contiene, en este caso el nombre de la columna y su tipo.

A continuación voy a usar el codigo "df_spark_sql.printSchema()" para obtener el esquema del dataframe.

In [14]:

print("----tipos de datos----")
for i in df_spark_sql.dtypes:
  print(i)
  
print("\n"*3)
print("----Esquema----")
df_spark_sql.printSchema()


----tipos de datos----
('Hotel_Address', 'string')
('Additional_Number_of_Scoring', 'int')
('Review_Date', 'string')
('Average_Score', 'double')
('Hotel_Name', 'string')
('Reviewer_Nationality', 'string')
('Negative_Review', 'string')
('Review_Total_Negative_Word_Counts', 'int')
('Total_Number_of_Reviews', 'int')
('Positive_Review', 'string')
('Review_Total_Positive_Word_Counts', 'int')
('Total_Number_of_Reviews_Reviewer_Has_Given', 'int')
('Reviewer_Score', 'double')
('Tags', 'string')
('days_since_review', 'int')
('lat', 'float')
('lng', 'float')
('score_string', 'string')
('score_evaluation', 'int')




----Esquema----
root
-- Hotel_Address: string (nullable = true)
-- Additional_Number_of_Scoring: integer (nullable = true)
-- Review_Date: string (nullable = true)
-- Average_Score: double (nullable = true)
-- Hotel_Name: string (nullable = true)
-- Reviewer_Nationality: string (nullable = true)
-- Negative_Review: string (nullable = true)
-- Review_Total_Negative_Word_Counts: integer (nullable = true)
-- Total_Number_of_Reviews: integer (nullable = true)
-- Positive_Review: string (nullable = true)
-- Review_Total_Positive_Word_Counts: integer (nullable = true)
-- Total_Number_of_Reviews_Reviewer_Has_Given: integer (nullable = true)
-- Reviewer_Score: double (nullable = true)
-- Tags: string (nullable = true)
-- days_since_review: integer (nullable = true)
-- lat: float (nullable = true)
-- lng: float (nullable = true)
-- score_string: string (nullable = true)
-- score_evaluation: integer (nullable = true)

### Ejercicio 2: Realizar un muestreo de 10 valores únicos de nombres de hoteles. Ordénalos alfanuméricamente de forma ascendente (primero los números 0-9, después A-Z). ###

Empiezo seleccionando la columna 'Hotel_Name'. A continuación le digo que solo seleccione los valores únicos. Posteriormente limito el numero de resultados a 10 y finalizo ordenando los resultados.

In [16]:
df_spark_sql.select('Hotel_Name').distinct().limit(10).orderBy('Hotel_Name').show()

+--------------------+
 Hotel_Name|
+--------------------+
 Avenida Palace|
Best Western Sera...|
Grange Holborn Hotel|
H tel Barri re Le...|
H tel Elysees Mermoz|
 H10 Port Vell 4 Sup|
 HCC Regente|
Hyatt Regency Ams...|
Melia Paris Notre...|
Melia Paris Tour ...|
+--------------------+

### Ejercicio 3: Transforma las columnas *lat* y *lng* al tipo Float.

Empiezo convirtiendo la columna "lat". Para dicha finalidad utilizo la función "witColumn" que crea una nueva columna,no obstante, dado que el nombre de la "nueva" columna es el mismo que una columna ya existente en el dataframe, la función usada lo que hará será sustituir la columna que figurava en el dataframe. 

A continuación hago referéncia a la columna 'df_spark_sql["lat"]' y termino diciendole que quiero que se convierta en tipo "Float". 

Termino aplicando el mismo procedimiento a la columna "lng"

In [18]:
df_spark_sql = df_spark_sql.withColumn("lat", df_spark_sql["lat"].cast("Float"))
df_spark_sql = df_spark_sql.withColumn("lng", df_spark_sql["lng"].cast("Float"))

In [19]:
splits = df_spark_sql.randomSplit([0.67, 0.33])
df_spark_sql_train = splits[0].dropna()
df_spark_sql_test = splits[1].dropna()
print(df_spark_sql_train.count())
print(df_spark_sql_test.count())

342967
169503

### Ejercicio 4: ¿Cuántos hoteles tienen una puntuación de 'Perfect'? ¿Y 'Good'? ¿Y 'Normal' junto a 'Good'? (Utilizar el dataset de Train)

Voy a empezar el ejercicio filtrando las filas de la columna "score_string" que contengan la palabra "Perfect". Como se puede ver, no hay ningún registro y esto lo compruebo con el comando "distinct". No figura ninguna entrada como "Perfect". 

A continuación aplico el mimso codigo para los registros en que figure la palabra "Good" y finalizo realizando lo mismo para "Normal" y "Good".

In [21]:
print("Puntuación 'Perfect'")
print(df_spark_sql_train.filter('score_string="Perfect"').count()) #no hay ninguno
df_spark_sql_train.select('score_string').distinct().show() #lo compruebo con el comando "distinct" y veo que efectivamente no hay ninguna valoración "Perfect"
print("\n","Puntuación 'Good'")
print(df_spark_sql_train.filter('score_string="Good"').count())
print("\n","Puntuación 'Good'+'Normal'")
print(df_spark_sql_train.filter('(score_string = "Normal") or (score_string = "Good")').count())
     

Puntuación 'Perfect'
0
+------------+
score_string|
+------------+
 Excellent|
 Good|
 Normal|
+------------+


 Puntuación 'Good'
285831

 Puntuación 'Good'+'Normal'
289583

### Ejercicio 5: Obtener los hoteles con mayor puntuación media, descartando todos los que tengan una puntuación por encima de Good. (Utilizar el dataset de Train) ###

Para hacer este ejercicio he empezado qutando las puntuaciones superiores a "Good" ("Perfect" y "Excellent"). 
Seguidamente he seleccionado las columnas que quería visualizar ('Hotel_Name',"Average_Score"). 
A continuación he agrupado los datos por "Hotel_Name" y he calculado la media de la "Average_Score". 
He proseguido con el comando "orderBy" para ordenar en orden descendente los hoteles por según sus medias (para conseguir los hoteles con la mayor puntuación). 
Finalmente he limitado los resultados a 10 para conseguir así lo 10 hoteles con mayor puntuación media sin incluir los hoteles que tengan una puntuación por encima de Good.

In [23]:
df_spark_sql_train.filter('(score_string <> "Perfect") and (score_string <> "Excellent")').select('Hotel_Name',"Average_Score").groupBy('Hotel_Name').avg("Average_Score").orderBy("avg(Average_Score)",ascending=False).limit(10).show()

+--------------------+------------------+
 Hotel_Name|avg(Average_Score)|
+--------------------+------------------+
Great Northern Ho...| 8.90000000000006|
The Marylebone Hotel| 8.900000000000059|
 Starhotels Echo| 8.900000000000059|
Negresco Princess...| 8.900000000000057|
Majestic Hotel Sp...| 8.900000000000052|
Hotel Okura Amste...| 8.90000000000005|
 K K Hotel Picasso| 8.900000000000048|
Col n Hotel Barce...| 8.900000000000045|
St Ermin s Hotel ...| 8.900000000000045|
Hotel Saint Peter...| 8.900000000000043|
+--------------------+------------------+

# Machine Learning en Apache Spark: Spark MLLib y Spark ML #

## Clasificación Supervisada: Árboles de decisión ##

### Ejercicio 6.1: Volver a observar todas las columnas del dataframe, para identificar las que sean categóricas. ###
Aplico el mismo codigo que usé en el primer ejercicio pero ahora con el dataframe "df_spark_sql_train" y veo que efectivamente existen veriables categoricas. 
Para efectuar los procesos de Machine learning deberemos eliminarlas o transformarlas en variables numéricas.

In [27]:
for i in df_spark_sql_train.dtypes:
  print(i)

('Hotel_Address', 'string')
('Additional_Number_of_Scoring', 'int')
('Review_Date', 'string')
('Average_Score', 'double')
('Hotel_Name', 'string')
('Reviewer_Nationality', 'string')
('Negative_Review', 'string')
('Review_Total_Negative_Word_Counts', 'int')
('Total_Number_of_Reviews', 'int')
('Positive_Review', 'string')
('Review_Total_Positive_Word_Counts', 'int')
('Total_Number_of_Reviews_Reviewer_Has_Given', 'int')
('Reviewer_Score', 'double')
('Tags', 'string')
('days_since_review', 'int')
('lat', 'float')
('lng', 'float')
('score_string', 'string')
('score_evaluation', 'int')

### Ejercicio 6.2: Eliminar, de los dataframes df_spark_sql_train y df_spark_sql test, las variables 'Hotel_Address', 'Hotel_Name', 'Tags', 'Positive Review', 'Negative_Review' y 'score_string'. Llamarlos: df_DT_train y df_DT_test. ### 

Optamos para eliminar las variables categoricas, menos "Review_Date" y "Review_Nationality" que transformaremos en variables numéricas.

In [29]:
df_DT_train = df_spark_sql_train.drop("Hotel_Address").drop("Hotel_Name")\
  .drop("Tags").drop("Positive_Review").drop("Negative_Review").drop("score_string")
df_DT_test = df_spark_sql_test.drop("Hotel_Address").drop("Hotel_Name")\
  .drop("Tags").drop("Positive_Review").drop("Negative_Review").drop("score_string")

for i in df_DT_train.dtypes:
  print(i)

('Additional_Number_of_Scoring', 'int')
('Review_Date', 'string')
('Average_Score', 'double')
('Reviewer_Nationality', 'string')
('Review_Total_Negative_Word_Counts', 'int')
('Total_Number_of_Reviews', 'int')
('Review_Total_Positive_Word_Counts', 'int')
('Total_Number_of_Reviews_Reviewer_Has_Given', 'int')
('Reviewer_Score', 'double')
('days_since_review', 'int')
('lat', 'float')
('lng', 'float')
('score_evaluation', 'int')

### Ejercicio 7: Para cada columa restante que sea String ('Review_Date' y 'Review_Nationality'), aplicar un StringIndexer(), devolviendo como resultado la misma columna, pero con su nombre acabando en _index. Sobreescribir ambos dataframes.  ###

En este ejercicio convierto los strings en numeros. Lo consigo pasando las columnas que continen caracteres por la función "StringIndexer", indicandole a dicha función la columna que quiero convertir y el nombre de la columna resultante. 
Con el "fit" coloco el dataframe dentro del cual se buscará la columna que quiero transformar para hacer los calculos pertinentes. Con el "transform" confirmo que quiero transformar los datos, generando así la columna "output". 

Este procedimiento lo aplico para las columnas de train y de test para las dos variables citadas en este ejercicio ('Review_Date' y 'Review_Nationality')

In [31]:

from pyspark.ml.feature import StringIndexer

df_DT_train = StringIndexer(inputCol="Review_Date", outputCol="Review_Date_index").fit(df_DT_train).transform(df_DT_train)
df_DT_train = StringIndexer(inputCol='Reviewer_Nationality', outputCol="Reviewer_Nationality_index").fit(df_DT_train).transform(df_DT_train)
df_DT_test = StringIndexer(inputCol="Review_Date", outputCol="Review_Date_index").fit(df_DT_test).transform(df_DT_test)
df_DT_test = StringIndexer(inputCol='Reviewer_Nationality', outputCol="Reviewer_Nationality_index").fit(df_DT_test).transform(df_DT_test)


for i in df_DT_train.dtypes:
  print(i)



('Additional_Number_of_Scoring', 'int')
('Review_Date', 'string')
('Average_Score', 'double')
('Reviewer_Nationality', 'string')
('Review_Total_Negative_Word_Counts', 'int')
('Total_Number_of_Reviews', 'int')
('Review_Total_Positive_Word_Counts', 'int')
('Total_Number_of_Reviews_Reviewer_Has_Given', 'int')
('Reviewer_Score', 'double')
('days_since_review', 'int')
('lat', 'float')
('lng', 'float')
('score_evaluation', 'int')
('Review_Date_index', 'double')
('Reviewer_Nationality_index', 'double')

### Ejercicio 8: Aplicar VectorAssembler() sobre las columnas que no son ni las dos anteriores, ni la columna 'score_evaluation', devolviendo una columna llamada 'features'. Llamar al resultado DT_vector_assembler. ###

En este ejercicio creo el Vector Assembler que dipositará toda la información que existe en el dataframe en un solo vector. 
Este paso es necesario ya que es la forma en que Spark ML espera obtener la información. 

Dado que hemos creado las variables "Review_Date_index" y "Reviewer_Nationality_index" le indicamos (mediante el comando drop) que en el vector no incluya las varaibles
"Review_Date" y "Reviewer_Nationality". 
Además, dado que "score_evaluation" es la columna que vamos a predecir, tampoco la incluimos.

Finalmente, le indico que el output (el vector que se creará) se le dé el nombre de "features".

In [33]:
from pyspark.ml.feature import VectorAssembler
LR_vector_assembler = VectorAssembler(\
  inputCols=df_DT_train.drop("Review_Date").drop("Reviewer_Nationality").drop("score_evaluation").columns,\
  outputCol="features")


### Ejercicio 9: Aplicar el transformador sobre ambos dataframes. ###
Ahora que ya se ha creado el "VectorAssembler" lo aplico con el "transform" y le doy los datos que se deben transfomar en vector, en ete caso el df_DT_train y el df_DT_test

In [35]:
df_DT_train = LR_vector_assembler.transform(df_DT_train)
df_DT_test = LR_vector_assembler.transform(df_DT_test)

### Ejercicio 10: Inicializar el modelo de árbol de decisión, entrenarlo y aplicarlo sobre los datos de test. ###
* Modelo: DecisionTreeClassifier:
  * Label: score_evaluation.
  * Features: features.
  * maxBins: 1000
  * maxDepth: 1
  
Dado que SparkMLLib requiere que hayamos creado los RDD y en este caso no lo hemos hecho, voy a utilizar las librerias de Spark ML. 

Voy a empezar importando las librerias necesarias.

A continuación voy a crear el modelo especificando el "label", el nombre de la columna con la que entrenaremos el modelo, así como la profundidad del modelo y los bins. 
Finalmente entreno el modelo pasando los datos del dataframe de entrenamiento.

Termino el ejercicio 10 aplicando el modelo entrenado a los datos que tengo guardados para el test.

In [37]:
from pyspark.ml.classification import DecisionTreeClassifier

model = DecisionTreeClassifier(labelCol="score_evaluation", featuresCol="features",maxDepth=1, maxBins=1000).fit(df_DT_train)

prediction = model.transform(df_DT_test)
print(prediction)

DataFrame[Additional_Number_of_Scoring: int, Review_Date: string, Average_Score: double, Reviewer_Nationality: string, Review_Total_Negative_Word_Counts: int, Total_Number_of_Reviews: int, Review_Total_Positive_Word_Counts: int, Total_Number_of_Reviews_Reviewer_Has_Given: int, Reviewer_Score: double, days_since_review: int, lat: float, lng: float, score_evaluation: int, Review_Date_index: double, Reviewer_Nationality_index: double, features: vector, rawPrediction: vector, probability: vector, prediction: double]

### Ejercicio 11: Evaluar el modelo aplicándole un clasificador multiclase. Calcular la métrica 'accuracy', y conseguir el complementario para calcular el error. ###
* Evaluador: MulticlassClassificationEvaluator
  * Label: score_evaluation.
  * Prediction: prediction.
  * MetricName: accuracy.

Voy a empezar este ejercicio importando las librerias que necesito, a continuación, creo el evaluador donde le indico que quiero realizar una predicción con "score_evaluation" y que estoy interesado en la métrica "accuracy".

Posteriormente al evaluador le paso la variable "prediction" que he conseguido en el ejercicio anterior para que calcule la "accuracy". 

Termino este ejercicio restando "1-accuracy" para saber el error existente.

In [39]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(
    labelCol="score_evaluation", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(prediction)
print(accuracy)
print(" Error = %g " % (1.0 - accuracy))

0.9887140640578633
 Error = 0.0112859

## Spark ML: Pipelines ##

### Pipelines: Árboles de Decisión ###
Con el mismo concepto que con el KMeans, se va a diseñar el flujo para los árboles de decisión. Primero hay que aplicar los cambios de preprocesamiento vistos anteriormente al DataFrame inicial para preparalo.

### Ejercicio 12: Eliminar, de los dataframes df_spark_sql_train y df_spark_sql test, las variables 'Hotel_Address', 'Hotel_Name', 'Tags', 'Positive Review', 'Negative_Review' y 'score_string'. Llamarlos: df_DT_train y df_DT_test. ### 

En este ejercicio aplico el mismo procedimiento que en el ejercicio 7.

In [43]:
df_DT_train = df_spark_sql_train.drop("Hotel_Address").drop("Hotel_Name")\
  .drop("Tags").drop("Positive_Review").drop("Negative_Review").drop("score_string")
df_DT_test = df_spark_sql_test.drop("Hotel_Address").drop("Hotel_Name")\
  .drop("Tags").drop("Positive_Review").drop("Negative_Review").drop("score_string")

Después se diseña el flujo para este modelo, el cual será:

** StringIndexer --> VectorAssembler --> Decission Tree (Inicialización) --> Decission Tree (Entrenamiento) --> Modelo Decission Tree entrenado **

### Ejercicio 13: Recoger una lista con todos los StringIndexer a aplicar, y llamarla DT_string_indexers ###
 En lugar de sobreescribir cada vez el dataframe, crear una lista, y con el método 'append', se irán añadiendo todos los StringIndexers().
 
 Empiezo con la creación de la Pipeline. 
 En este caso le digo que si el tipo de la columna es igual a "string" que cree una variable "StringIndexer" que se guarda en la variable "DT_string_indexers" para poder montar correctamente la pipeline.

In [46]:
DT_string_indexers = []
for dtype in df_DT_train.dtypes:
  if dtype[1] == "string":
    DT_string_indexers.append(StringIndexer(inputCol=dtype[0], outputCol=dtype[0]+"_index"))

### Ejercicio 14: Guardar en la variable 'DT_vector_assembler' la aplicación del mismo VectorAssembler() del ejercicio 8. ###
Hago lo mismo que en el ejercicio anterior, para montar la pipeline, creo una variable que contiene el "VectorAssembler" guardando el resultado en la variable "D_vector_assembler"

In [48]:
DT_vector_assembler = VectorAssembler(\
  inputCols=df_DT_train.drop("Review_Date").drop("Reviewer_Nationality").drop("score_evaluation").columns,\
  outputCol="features")

### Ejercicio 15: Crear una lista con el mombre de DT_pipeline_stages, y añadirle la lista de StringIndexers y el VectorAssembler (en este orden) ###

En este ejercicio simplemente diposito la información de las variables que he creado en los dos últimos ejercicios en la "DT_pipeline_stages".

In [50]:
DT_pipeline_stages = [str_indexer for str_indexer in DT_string_indexers]
DT_pipeline_stages.append(DT_vector_assembler)

### Ejercicio 16: Inicializar el modelo de árbol de decisión (mismas especificaciones que en el ej. 10), y añadirlo a la lista de pasos 'DT_pipeline_stages' ###

Dentro de la pipeline también estará el modelo creado. Esta vez, no lo entrenamos (como sí que hicimos en le ejercicio 10), ya que lo haremos en el último paso.

In [52]:
model = DecisionTreeClassifier(labelCol="score_evaluation", featuresCol="features",maxDepth=1, maxBins=1000)
DT_pipeline_stages.append(model)

### Ejercicio 17: Diseñar el Pipeline y aplicarlo sobre los datos de Train, llamándolo 'DT_pipeline_model' ###
En este ejercicio pasamos toda la pipeline creada por la función Pipeline. Esto crea un objeto que aplicaremos en el próximo ejercicio.

In [54]:
from pyspark.ml import Pipeline
DT_pipeline = Pipeline(stages=DT_pipeline_stages)
print(DT_pipeline)

Pipeline_1c30159ce883

### Ejercicio 18: Aplicar el modelo resultante sobre los datos de test y evaluarlo al igual que se hizo en el ej. 11 ###

Finalmente, introduzco los datos a la pipeline, esta, hará todos pasos que hemos dipositado dentro de la misma, finalizando con el proceso del "training" del modelo. 

Termino el ejercicio calculando la "accuracy" del modelo con los datos de dataframe del test.

In [56]:
DT_pipeline_model = DT_pipeline.fit(df_DT_train)

prediction = DT_pipeline_model.transform(df_DT_test)
evaluator = MulticlassClassificationEvaluator(
    labelCol="score_evaluation", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(prediction)
print(accuracy)
print(" Error = %g " % (1.0 - accuracy))

0.9887140640578633
 Error = 0.0112859